In [76]:
from rdflib import Graph
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import random

In [64]:
input_path = 'sto/'
kg = input_path + 'sto-enriched.nt'
dataset = pd.read_csv(kg, delimiter='---', header=None)
dataset.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


,0
0,<http://www.w3.org/1999/02/22-rdf-syntax-ns#va...
1,<http://www.w3.org/2001/XMLSchema#gYear> <http...
2,<http://www.w3.org/2001/XMLSchema#date> <http:...
3,<http://purl.org/dc/elements/1.1/relation> <ht...
4,<http://purl.org/dc/elements/1.1/description> ...


In [65]:
g = Graph()
g.parse("sto/sto-enriched.nt", format="nt")

qres = g.query(
    """PREFIX owl: <http://www.w3.org/2002/07/owl#>
       PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
       PREFIX sto: <https://w3id.org/i40/sto#>
    
    select ?s where {
            ?s rdf:type sto:Standard .
    } limit 1000""")
        
entities = pd.DataFrame(columns=['s'])
obj = []
for row in qres:
    obj.append(str("%s" %row))
entities.s=obj

## Select all standard triples given a specific property

In [66]:
count = 0
prop = 'hasClassification'
#prop = 'relatedTo'
list_pd = pd.DataFrame(columns=['s', 'p', 'o'])
for i in range(entities.shape[0]):
    standard = entities.s[i]
    qres = g.query(
        """PREFIX owl: <http://www.w3.org/2002/07/owl#>
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            PREFIX sto: <https://w3id.org/i40/sto#>
            select distinct ?o where {
                <""" +standard+ """> sto:""" +prop+ """ ?o .
            }""")
    lst = pd.DataFrame(columns=['s', 'p', 'o'])
    obj = []
    for row in qres:
        obj.append('<'+str("%s" %row)+'>')
    
    if len(obj)>0:
        lst.o=obj
        lst.s='<'+standard+'>'
        lst.p='<https://w3id.org/i40/sto#'+prop+'>'
        list_pd = pd.concat([list_pd, lst], ignore_index=True)
    else:
        count+=1

#list_pd = list_pd.drop_duplicates().reset_index(drop=True)

## Create the test set [Select 30% of triple randomly] 

In [67]:
n_select= round(3*list_pd.shape[0]/10)
print(n_select)
index = list_pd.index
#sampling = random.choices(index, k=n_select)
sampling = random.sample(index, k=n_select)

102


In [68]:
test_set=list_pd.iloc[sampling]
duplicateRowsDF = test_set[test_set.duplicated()]
test_set = test_set.drop_duplicates().reset_index(drop=True)
print(test_set.shape[0])
duplicateRowsDF

88


,s,p,o
284,<https://w3id.org/i40/sto#IEC_62541>,<https://w3id.org/i40/sto#hasClassification>,<https://w3id.org/i40/sto#IICFFrameworkLayer>
299,<https://w3id.org/i40/sto#NIST_800-146>,<https://w3id.org/i40/sto#hasClassification>,<https://w3id.org/i40/sto#IIRASecurity>
36,<https://w3id.org/i40/sto#ISO_24760>,<https://w3id.org/i40/sto#hasClassification>,<https://w3id.org/i40/sto#ProductModelDataExch...
244,<https://w3id.org/i40/sto#ISO_14306>,<https://w3id.org/i40/sto#hasClassification>,<https://w3id.org/i40/sto#RAMIAssetlayer>
289,<https://w3id.org/i40/sto#IEC_62541>,<https://w3id.org/i40/sto#hasClassification>,<https://w3id.org/i40/sto#IDSRAMSystemLayer>
145,<https://w3id.org/i40/sto#IEC_62714>,<https://w3id.org/i40/sto#hasClassification>,<https://w3id.org/i40/sto#ProductionSystemEngi...
153,<https://w3id.org/i40/sto#ISO_13849>,<https://w3id.org/i40/sto#hasClassification>,<https://w3id.org/i40/rami#Safety>
63,<https://w3id.org/i40/sto#RFC_7643>,<https://w3id.org/i40/sto#hasClassification>,<https://w3id.org/i40/sto#FiwareSecurity>
19,<https://w3id.org/i40/sto#ISO_15704>,<https://w3id.org/i40/sto#hasClassification>,<https://w3id.org/i40/sto#RAMIBusinessLayer>
4,<https://w3id.org/i40/sto#IEC_62337>,<https://w3id.org/i40/sto#hasClassification>,<https://w3id.org/i40/rami#Lifecycle_Status>


In [ ]:
#test_set.sort_values(by='s', ascending=False)

In [69]:
test_set['dot'] = '.'

In [70]:
test_set_nt=pd.DataFrame(test_set['s'] + ' '+ test_set['p']+ ' '+ test_set['o']+ ' '+ test_set['dot'])
test_set_nt.head(20)

,0
0,<https://w3id.org/i40/sto#DIN_EN_ISO_IEC_17025...
1,<https://w3id.org/i40/sto#ISO_9594> <https://w...
2,<https://w3id.org/i40/sto#IEC_62541> <https://...
3,<https://w3id.org/i40/sto#IEC_61804> <https://...
4,<https://w3id.org/i40/sto#ISO_14306> <https://...
5,<https://w3id.org/i40/sto#IEC_24760> <https://...
6,<https://w3id.org/i40/sto#ISO_8062-4> <https:/...
7,<https://w3id.org/i40/sto#IEC_62541> <https://...
8,<https://w3id.org/i40/sto#IEC_62541> <https://...
9,<https://w3id.org/i40/sto#NIST_800-146> <https...


## Create training test [difference bettwen dataset and test set]

In [71]:
training_set = dataset[~dataset.apply(tuple,1).isin(test_set_nt.apply(tuple,1))]
training_set.shape[0]

21061

In [72]:
dataset.shape[0]-training_set.shape[0]

88

## Save training and test set

In [73]:
training_set.to_csv(input_path+'training_set_hasClassification.nt', index=None, header=None)
test_set_nt.to_csv(input_path+'test_set_hasClassification.nt', index=None, header=None)

## Cheking the training set created

In [74]:
g = Graph()
g.parse("sto/training_set_hasClassification.nt", format="nt")
    
qres = g.query(
    """PREFIX owl: <http://www.w3.org/2002/07/owl#>
       PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
       PREFIX sto: <https://w3id.org/i40/sto#>
    
    select ?s where {
            ?s rdf:type sto:Standard .
    } limit 1000""")
        
entities = pd.DataFrame(columns=['s'])
obj = []
for row in qres:
    obj.append(str("%s" %row))
entities.s=obj

In [75]:
entities

,s
0,https://w3id.org/i40/sto#IEC_60904
1,https://w3id.org/i40/sto#IEC_61000_P6_S2_E3
2,https://w3id.org/i40/sto#IEC_60870_P5_S104_E2_1
3,https://w3id.org/i40/sto#UDP
4,https://w3id.org/i40/sto#IEC_60870_P5_S1_E1
5,https://w3id.org/i40/sto#NIST_800-144
6,https://w3id.org/i40/sto#ISO_14306
7,https://w3id.org/i40/sto#ISO_19770
8,https://w3id.org/i40/sto#IEC_60839_P7_S1_E1
9,https://w3id.org/i40/sto#IEC_61310_P3_E2
